In [25]:
import bs4
import urllib
import operator
from bs4 import BeautifulSoup as soup

myURL = 'https://exvius.gamepedia.com/Rain'
# Set up connection to the desire URL and opening the page
req = urllib.request.Request(myURL, headers={'User-Agent' : "Magic Browser"}) 
con = urllib.request.urlopen(req)
htmlpage = con.read()

# Close the connection
con.close()

minRarity = 0
maxRarity = 0

# Hold values for resistances
resists = {
    "fire": 0,
    "ice": 0,
    "lightning": 0,
    "water": 0,
    "wind": 0,
    "earth": 0,
    "light": 0,
    "dark": 0,
    "poison": 0,
    "blind": 0,
    "sleep": 0,
    "silence": 0,
    "paralysis": 0,
    "confuse": 0,
    "disease": 0,
    "petrification": 0
}

# Hold values for equipment usage
equippable = {
    "dagger": "false",
    "sword": "false",
    "great sword": "false",
    "katana": "false",
    "staff": "false",
    "rod": "false",
    "bow": "false",
    "axe": "false",
    "hammer": "false",
    "spear": "false",
    "harp": "false",
    "whip": "false",
    "throwing weapon": "false",
    "gun": "false",
    "mace": "false",
    "fist": "false",
    "light shield": "false",
    "heavy shield": "false",
    "hat": "false",
    "helm": "false",
    "clothes": "false",
    "light armor": "false",
    "heavy armor": "false",
    "robe": "false",
    "accessory": "false"
}

# Parse the HTML file using BeautifulSoup
page_soup = soup(htmlpage, "html.parser")

f = open('unitdata.txt', 'w')
# Open JSON
f.write("{\n")

wikitable = page_soup.find('table', class_='wikitable')

wikitable_info = wikitable.find_all('tr')

# Get the unit name
name = wikitable_info[0].th.text.rstrip()

rarities = wikitable_info[2].td.text

# Get the minimum rarity of the unit
minRarity = rarities.split(' ')[0].split('-')[0]

# Get the maximum rarity of the unit
maxRarity = rarities.split(' ')[2].split('-')[0]

# Get the TMR of the unit
tmr = wikitable_info[12].td.text.split('\n')[0]
tmr = removeRepeatedSubstring(tmr)

f.write('"name": "%s",\n' % name)
f.write('"base rarity": "%s",\n' % minRarity)
f.write('"max rarity": "%s", \n' % maxRarity)
f.write('"tmr": "%s", \n' % tmr);

# If the unit has a 7 star awakening, get the STMR of the unit
if(maxRarity == "7"):
    stmr = wikitable_info[13].td.text
    stmr = removeRepeatedSubstring(stmr)
    f.write('"stmr": "%s",\n' % stmr)

stats_list = []

# Get the base stats of the unit
stats = page_soup.find_all('table', class_='wikitable')
base = stats[1].find_all('tr')

for i in range(0,len(base)):
    if(i == 0):
        continue
    
    base_hp = base[i].text.split('\n')[2]
    stats_list.append('"hp":' + '"' + base_hp)
    base_mp = base[i].text.split('\n')[3]
    stats_list.append('"mp":' + '"' + base_mp)
    base_atk = base[i].text.split('\n')[4]
    stats_list.append('"atk":' + '"' + base_atk)
    base_def = base[i].text.split('\n')[5]
    stats_list.append('"def":' + '"' + base_def)
    base_mag = base[i].text.split('\n')[6]
    stats_list.append('"mag":' + '"' + base_mag)
    base_spr = base[i].text.split('\n')[7]
    stats_list.append('"spr":' + '"' + base_spr)

# Get the pot increase stats of the unit
inc = stats[2].find_all('tr')
counter = 0
for i in range(0, len(inc)):
    if(i == 0):
        continue
    
    inc_hp = inc[i].text.split('\n')[2]
    stats_list[counter] = stats_list[counter] + " (+" + inc_hp + ')",'
    counter += 1
    inc_mp = inc[i].text.split('\n')[3]
    stats_list[counter] = stats_list[counter] + " (+" + inc_mp + ')",'
    counter += 1
    inc_atk = inc[i].text.split('\n')[4]
    stats_list[counter] = stats_list[counter] + " (+" + inc_atk + ')",'
    counter += 1
    inc_def = inc[i].text.split('\n')[5]
    stats_list[counter] = stats_list[counter] + " (+" + inc_def + ')",'
    counter += 1
    inc_mag = inc[i].text.split('\n')[6]
    stats_list[counter] = stats_list[counter] + " (+" + inc_mag + ')",'
    counter += 1
    inc_spr = inc[i].text.split('\n')[7]
    if((counter+1) % 6 == 0):
        stats_list[counter] = stats_list[counter] + " (+" + inc_spr + ')"'
    else:
        stats_list[counter] = stats_list[counter] + " (+" + inc_spr + ')",'
    counter += 1
    
f.write('"stats": [\n{\n') 
for i in range(0, len(stats_list)):
    f.write(stats_list[i])
    f.write('\n')
    if((i+1) == len(stats_list)):
        f.write('}\n')
    elif((i+1) % 6 == 0):
        f.write('},\n{\n')
f.write("],\n")

# Get the resistances of the unit
resist = page_soup.find_all('table', class_='wikitable')
elem_resist = resist[3].find_all('tr')
resist_itr = 0

f.write('"resistances": [\n{\n')

for image in elem_resist[1].find_all('img'):
    elem = image.get('alt','').lower().split(' ')[0]
    resists[elem] = elem_resist[2].find_all('td')[resist_itr].text.split("\n")[0]
    resist_itr += 1

ailment_resist = resist[3].find_all('tr')

resist_itr = 0
for image in ailment_resist[4].find_all('img'):
    ailment = image.get('alt', '').lower().split(' ')[0]
    resists[ailment] = ailment_resist[5].find_all('td')[resist_itr].text.split("\n")[0]
    resist_itr += 1

for key,val in resists.items():
    f.write('"%s": "%s"' %(key, val))
    if(key == "petrification"):
        f.write("\n")
    else:
        f.write(",\n")
f.write('}\n],\n')

# Get the equipment that the unit can use
equip = page_soup.find_all('table', class_='wikitable')
usable = equip[4].find_all('tr')

for image in usable[1].find_all('img'):
    eqp = image.get('alt', '').lower()
    equippable[eqp] = "true"

for image in usable[3].find_all('img'):
    eqp = image.get('alt', '').lower()
    equippable[eqp] = "true"

f.write('"equipment": [\n{\n')

for key,val in equippable.items():
    f.write('"%s": %s' %(key, val.lower()))
    if(key == "accessory"):
        f.write("\n")
    else:
        f.write(",\n")

f.write('}\n],\n')

all_skills = []

# Get the unit's special skills (actives and passives), if any
special = page_soup.find('span', id='Special')
if(special is not None):
    h3 = special.find_parent('h3')
    active = h3.find_next_sibling('table')
    skill = active.find_all('tr')
    
    for i in range(2,len(skill)):
        if(skill[i].text.split('\n')[1] == 'Conditional'):
            # Find out what to do with the conditional skills
            break
        elif(skill[i].text.split('\n')[1] == 'Passive ' or skill[i].text.split('\n')[1] == 'Min rarity'):
            continue
        else:
            if (len(skill[i].text.split('\n')) == 15):
                skill_rarity = int(skill[i].text.split('\n')[1].split('-')[0])
                skill_level = int(skill[i].text.split('\n')[3])
                skill_name = skill[i].text.split('\n')[7]
                desc = skill[i].text.split('\n')[9]
                descCopy = ""
                for ind in range(0, len(desc)):
                    if((ind + 1) < len(desc) and (desc[ind].islower() or desc[ind] == ')')
                       and desc[ind+1].isupper()):
                        descCopy = descCopy + desc[ind]
                        descCopy = descCopy + "<br />"              
                    else:
                        descCopy = descCopy + desc[ind]
                skill_mp = skill[i].text.split('\n')[13]
                
                # Create a tuple with this info and insert it into a list for sorting
                skill_info = (skill_rarity, skill_level, skill_name, "Active", skill_mp, descCopy)
                all_skills.append(skill_info)
                
            else:
                skill_rarity = int(skill[i].text.split('\n')[1].split('-')[0])
                skill_level = int(skill[i].text.split('\n')[3])
                skill_name = skill[i].text.split('\n')[7]
                desc = skill[i].text.split('\n')[9]
                descCopy = ""
                for ind in range(0, len(desc)):
                    if((ind + 1) < len(desc) and (desc[ind].islower() or desc[ind] == ')')
                       and desc[ind+1].isupper()):
                        descCopy = descCopy + desc[ind]
                        descCopy = descCopy + "<br />"              
                    else:
                        descCopy = descCopy + desc[ind]
                #print('Description: %s' % descCopy)
                skill_mp = "-"
                skill_info = (skill_rarity, skill_level, skill_name, "Passive", skill_mp, descCopy)
                all_skills.append(skill_info)
                
# Get the unit's magic skills, if any
magic = page_soup.find('span', id='Magic')

if(magic is not None):
    h3 = magic.find_parent('h3')
    mags = h3.find_next_sibling('table')
    magic_skill = mags.find_all('tr')
    
    for i in range(1,len(magic_skill)):
        skill_rarity = int(magic_skill[i].text.split('\n')[1].split('-')[0])
        skill_level = int(magic_skill[i].text.split('\n')[3])
        skill_name = magic_skill[i].text.split('\n')[7]
        desc = magic_skill[i].text.split('\n')[9]
        descCopy = ""
        for ind in range(0, len(desc)):
            if((ind + 1) < len(desc) and (desc[ind].islower() or desc[ind] == ')')
               and desc[ind+1].isupper()):
                descCopy = descCopy + desc[ind]
                descCopy = descCopy + "<br />"              
            else:
                descCopy = descCopy + desc[ind]
        skill_mp = magic_skill[i].text.split('\n')[13]
        skill_info = (skill_rarity, skill_level, skill_name, "Magic", skill_mp, descCopy)
        all_skills.append(skill_info)

# Sort the skills first by rarity and level

all_skills.sort(key=operator.itemgetter(1))
all_skills.sort(key=operator.itemgetter(0))

conditional = page_soup.find()

f.write('"abilities": [\n{\n')
for i in range(0, len(all_skills)):
    
    f.write('"rarity": %s,\n' %(all_skills[i][0]))
    f.write('"level": %s,\n' %(all_skills[i][1]))
    f.write('"name": "%s",\n' %(all_skills[i][2]))
    f.write('"type": "%s",\n' %(all_skills[i][3]))
    f.write('"effect": "%s",\n' %(all_skills[i][4]))
    f.write('"mp": "%s"\n' %(all_skills[i][5]))
    
    if((i+1) == len(all_skills)):
        f.write('}\n')
    else:
        f.write('},\n{\n')
    
f.write(']\n')
# Get any ability awakenings for this unit
awakening = page_soup.find('span', id='Ability_Awakening')

if(awakening is not None):
    f.write(',\n')
    f.write('"ability awakenings": [\n')
    h3 = awakening.find_parent('h3')
    awak_table = h3.find_next_sibling('table')
    a_skills = awak_table.find_all('tr')
    
    for i in range(1, len(a_skills)):
        if(a_skills[i].has_attr("class")):
            continue
        f.write('{\n')
        a_skill_name = a_skills[i].text.split('\n')[1]
        f.write('"name": "%s",\n' %(a_skill_name))
        a_skill_desc = a_skills[i].text.split('\n')[3]
        descCopy = ""
        for ind in range(0, len(a_skill_desc)):
            if((ind + 1) < len(a_skill_desc) and (a_skill_desc[ind].islower() or a_skill_desc[ind] == ')') 
               and a_skill_desc[ind+1].isupper()):
                descCopy = descCopy + a_skill_desc[ind]
                descCopy = descCopy + "<br />"              
            else:
                descCopy = descCopy + a_skill_desc[ind]
        f.write('"description": "%s",\n' %(descCopy))
        a_skill_mp = a_skills[i].text.split('\n')[7]
        f.write('"mp": "%s",\n' %(a_skill_mp))
        a_skill_cryst = a_skills[i].text.split('\n')[9]
        f.write('"type": "%s",\n' %(a_skill_cryst))
        a_skill_t1 = a_skills[i].text.split('\n')[11]
        f.write('"t1": "%s",\n' %(a_skill_t1))
        a_skill_t2 = a_skills[i].text.split('\n')[13]
        f.write('"t2": "%s",\n' %(a_skill_t2))
        a_skill_t3 = a_skills[i].text.split('\n')[15]
        f.write('"t3": "%s",\n' %(a_skill_t3))
        a_skill_t4 = a_skills[i].text.split('\n')[17]
        f.write('"t4": "%s",\n' %(a_skill_t4))
        a_skill_t5 = a_skills[i].text.split('\n')[19]
        f.write('"t5": "%s",\n' %(a_skill_t5))
        a_skill_gil = a_skills[i].text.split('\n')[21]
        f.write('"gil": "%s"\n' %(a_skill_gil))
        if(i == len(a_skills)-1):
            f.write('}\n')
        else:
            f.write('},\n')
    f.write(']\n')
# Close JSON
f.write('}')
f.close()

# Get the materia slots and/or usable magic of the unit
materia = page_soup.find_all('table', class_='wikitable')
slots = materia[4].find_all('tr')
affinities = []
itr = 0

for i in range(5, len(slots)):
    rarity = slots[i].text.split('\n')[1].split('-')[0]
    num_slots = slots[i].text.split('\n')[2]
    magic_lvl = slots[i].text.split('\n')[3].split(',')

    for j in range(0, len(magic_lvl)):
        affinity = magic_lvl[j].rstrip()
        affinities.append(affinity)
    for img in slots[i].find_all('img'):
        img_data = img.get('alt','')
        first_char = img_data[0]
        # Already have the rarities from previous parse, so skip
        if(first_char != 'W' and first_char != 'B' and first_char != 'G'):
            continue
        else:
            affinities[itr] = rarity + ' ' + num_slots + ' ' + img_data.split(' ')[0] + affinities[itr]
            itr += 1
    
print(affinities)
            

['2 1 White Lvl 1', '2 1 Black  Lvl 2', '3 2 White Lvl 2', '3 2 Black  Lvl 3', '4 2 White Lvl 3', '4 2 Black  Lvl 4', '5 3 White Lvl 4', '5 3 Black  Lvl 6', '6 4 White Lvl 5', '6 4 Black  Lvl 7']


In [2]:
import math

# Function to remove the tooltip information when scraping the Trust Master Reward (TMR) info
def removeRepeatedSubstring(tmrString):
    tmr = set()
    copyString = ""
    for i in range(0, len(tmrString)):
        if(tmrString[i] not in tmr):
            tmr.add(tmrString[i])
        elif(tmrString[i] in tmr and tmrString[i] == tmrString[0]):
            firstHalf = tmrString[0:math.floor((i/2))]
            secondHalf = tmrString[math.floor((i/2)): i]
            if(firstHalf == secondHalf):
                return firstHalf   
